In [14]:
import pandas as pd
import os

def compare_files(current_file, previous_file, current_cols, previous_cols, output_directory):
    
    """
    Compares two CSV or excel files based on specified columns and writes the comparison results to an output file.
    The function generates hash values for each row in both files based on the specified columns.
    It identifies rows present in both files, rows present only in the current file, and rows present only in the previous file.
    The comparison results are saved in a text file in the specified output directory.
    
    Args:
        current_file (str): Path to the current CSV file.
        previous_file (str): Path to the previous CSV file.
        current_cols (list): List of column names to compare in the current CSV file.
        previous_cols (list): List of column names to compare in the previous CSV file.
        output_directory (str): Directory where the output file will be saved.
        
    Returns:
        None
    """
    
    
    # Determine file format and read appropriately
    ext_current = os.path.splitext(current_file)[1].lower()
    ext_previous = os.path.splitext(previous_file)[1].lower()

    if ext_current == '.csv':
        df1 = pd.read_csv(current_file)
    elif ext_current in ['.xls', '.xlsx']:
        df1 = pd.read_excel(current_file)
    else:
        raise ValueError("Unsupported format for current file.")

    if ext_previous == '.csv':
        df2 = pd.read_csv(previous_file)
    elif ext_previous in ['.xls', '.xlsx']:
        df2 = pd.read_excel(previous_file)
    else:
        raise ValueError("Unsupported format for previous file.")
    
    # strip leading/trailing spaces from column names
    df1.columns = df1.columns.str.strip()
    df2.columns = df2.columns.str.strip()

    # Define keywords to convert columns to string (case-insensitive) in case they are not defined correctly
    keywords = [] # Update based on your dataset or leave it empty

    # Convert specific columns to string if their names contain certain keywords (case-insensitive)
    for col in df1.columns:
        if any(keyword in col.lower() for keyword in keywords):
            df1[col] = df1[col].astype(str)
            print(f"Column '{col}' in Current File converted to string.")
    
    for col in df2.columns:
        if any(keyword in col.lower() for keyword in keywords):
            df2[col] = df2[col].astype(str)
            print(f"Column '{col}' in Previous File converted to string.")

    # Strip extra spaces, collapse multiple spaces between words to a single space, and convert strings to lowercase
    df1 = df1.applymap(lambda x: ' '.join(x.strip().split()) if isinstance(x, str) else x)
    df1 = df1.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    df2 = df2.applymap(lambda x: ' '.join(x.strip().split()) if isinstance(x, str) else x)
    df2 = df2.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    # Ensure that the user-selected columns are present in both files
    common_cols_in_both_files = list(set(df1.columns) & set(df2.columns))

    if not common_cols_in_both_files:
        print("No common columns found between the two files.")
        return

    print(f"Common columns for comparison: {common_cols_in_both_files}")
    
    common_cols_in_both_files = [col for col in df1.columns if col in common_cols_in_both_files]
    df1 = df1[common_cols_in_both_files]
    df2 = df2[common_cols_in_both_files]

    # Generate hashes for rows based on specified columns
    df1['hash'] = df1[current_cols].apply(lambda x: hash(tuple(x)), axis=1)
    df2['hash'] = df2[previous_cols].apply(lambda x: hash(tuple(x)), axis=1)

    # Extract unique hashes from each file
    unique_hashes_df1 = set(df1['hash'])
    unique_hashes_df2 = set(df2['hash'])

    # Identify rows present only in one file or both files
    rows_present_in_both_files = unique_hashes_df1.intersection(unique_hashes_df2)
    rows_present_only_in_df1 = unique_hashes_df1 - unique_hashes_df2
    rows_present_only_in_df2 = unique_hashes_df2 - unique_hashes_df1

    # Append results: output lines
    output_lines = []
    output_lines.append("\nNote:")
    output_lines.append(f"\nCurrent File: {current_file}\n")
    output_lines.append("Columns in Current File:")
    for col in df1.columns:
        output_lines.append(f"- {col}")
    output_lines.append("")
    
    output_lines.append(f"\nPrevious File: {previous_file}\n")
    output_lines.append("Columns in Previous File:")
    for col in df2.columns:
        output_lines.append(f"- {col}")
    output_lines.append("\n")
    
    # Define the threshold for significant percent change
    THRESHOLD = 50  # You can adjust this value as needed
    
    # Iterate through rows present in both files
    for index, current_row in df1.iterrows():
        hash_value = hash(tuple(current_row[current_cols]))
        if hash_value in rows_present_in_both_files:
            # Row present in both files
            previous_row = df2[df2['hash'] == hash_value].iloc[0]
            percent_changes = []
            has_non_zero_change = False

            # Calculate percent change for numeric columns
            for idx, (current_val, previous_val) in enumerate(zip(current_row.values, previous_row.values)):
                # Ensure both current and previous values are numeric
                if pd.api.types.is_numeric_dtype(df1.iloc[:, idx]) and pd.api.types.is_numeric_dtype(df2.iloc[:, idx]):
                    # Coerce to numeric and handle errors
                    current_val = pd.to_numeric(current_val, errors='coerce')
                    previous_val = pd.to_numeric(previous_val, errors='coerce')

                    # Ensure we skip NaN values
                    if pd.notnull(current_val) and pd.notnull(previous_val):
                        if previous_val != 0:
                            percent_change = ((current_val - previous_val) / previous_val) * 100
                        else:
                            if current_val == 0:
                                percent_change = 0.0  # Both current and previous values are zero, so percent change is zero
                            else:
                                percent_change = float('inf')  # Handle division by zero if previous value is zero

                        # Collect percent change values if they are not zero or infinity
                        if abs(percent_change) >= THRESHOLD and percent_change != float('inf'):
                            column_name = df1.columns[idx]
                            percent_changes.append(f"Percent Change at Column '{column_name}' (Index {idx}): {percent_change:.2f}%\n")
                            has_non_zero_change = True

            # Append rows with at least one non-zero percent change
            if has_non_zero_change:
                output_lines.append("\nCurrent File row : {}\n".format(", ".join(map(str, current_row.values))))
                output_lines.append("Previous File row  : {}\n".format(", ".join(map(str, previous_row.values))))
                output_lines.extend(percent_changes)
                output_lines.append("\n--- End of Comparison ---\n")
                
    # Output rows present only in the current file
    output_lines.append ("\n"+ "="*100)
    output_lines.append("\nRows present only in the current file:\n")    
    for hash_value in rows_present_only_in_df1:
        current_row = df1[df1['hash'] == hash_value].iloc[0]
        output_lines.append("Current File row  : {}\n".format(", ".join(map(str, current_row.values))))

    # Output rows present only in the previous file
    output_lines.append ("\n"+ "="*100)
    output_lines.append("\nRows present only in the previous file:\n")
    for hash_value in rows_present_only_in_df2:
        previous_row = df2[df2['hash'] == hash_value].iloc[0]
        output_lines.append("Previous File row  : {}\n".format(", ".join(map(str, previous_row.values))))

    # Write output to a file
    output_text = ''.join(output_lines)
    current_filename = os.path.basename(current_file)
    previous_filename = os.path.basename(previous_file)
    output_file_name = f'{os.path.splitext(current_filename)[0]}_vs_{os.path.splitext(previous_filename)[0]}_output.txt'
    output_path = os.path.join(output_directory, output_file_name)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(output_text)

    print("Comparison completed. Results written to", output_path)


# Usage example
path1 = "C:\\Users\\hayet.chekired.ctr\\Documents\\Learning\\File Comparison Script\\Test_1"
path2 = "C:\\Users\\hayet.chekired.ctr\\Documents\\Learning\\File Comparison Script\\Test_2"

current_file = os.path.join(path1, "Clean_Table-2-5303-and-5304-FY25-Full-Year.csv")
previous_file = os.path.join(path2, "Clean_Table-2-5303-and-5304-FY-2024-Full-Year.csv")

current_cols = ["STATE"]
previous_cols = ["STATE"]

output_directory = "output_txt"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Perform file comparison and save the results
compare_files(current_file, previous_file, current_cols, previous_cols, output_directory)


Common columns for comparison: ['SECTION 5305(d)\n5303 APPORTIONMENT', 'SECTION 5305(e)\n5304 APPORTIONMENT', 'STATE']
Comparison completed. Results written to output_txt\Clean_Table-2-5303-and-5304-FY25-Full-Year_vs_Clean_Table-2-5303-and-5304-FY-2024-Full-Year_output.txt
